In [1]:
import os, re

# adjust these to wherever your folders actually live
folders = {
    "images": "yolo_dtaa/Object Detection.v1i.yolov5pytorch/train/images",   # e.g. "data/images"
    "labels": "yolo_dtaa/Object Detection.v1i.yolov5pytorch/train/labels",   # e.g. "data/labels"
}

# this regex grabs everything up to "_png.rf.<hash>" as group(1),
# and preserves the extension (e.g. ".jpg" or ".txt") as group(2)
pattern = re.compile(r"^(.+?)_png\.rf\.[0-9a-fA-F]+(\.[^.]+)$")

for kind, folder in folders.items():
    print(f"\nProcessing {kind} in `{folder}`:")
    for fname in os.listdir(folder):
        m = pattern.match(fname)
        if not m:
            print(f"  • skipping (no match): {fname}")
            continue

        base_id   = m.group(1)      # e.g. "20250718_113752_269634"
        ext       = m.group(2)      # e.g. ".jpg" or ".txt"
        new_name  = base_id + ext

        src = os.path.join(folder, fname)
        dst = os.path.join(folder, new_name)
        os.rename(src, dst)
        print(f"  • {fname} → {new_name}")



Processing images in `yolo_dtaa/Object Detection.v1i.yolov5pytorch/train/images`:
  • 20250718_113752_269634_png.rf.037f6148a2406b4bdc3d51474ca38659.jpg → 20250718_113752_269634.jpg
  • 20250718_113754_714255_png.rf.3d0d3afa487db62455e4eedf03f1fa2c.jpg → 20250718_113754_714255.jpg
  • 20250718_113755_102432_png.rf.772502200e7be0788c025e9c6c8cf4fc.jpg → 20250718_113755_102432.jpg
  • 20250718_113755_810809_png.rf.90ebc8a5f05c2f44a135cea7cd7e2c51.jpg → 20250718_113755_810809.jpg
  • 20250718_113756_476592_png.rf.31d71d0ae2b8d7c51dbe1c9cf0e87c34.jpg → 20250718_113756_476592.jpg
  • 20250718_113757_311472_png.rf.39a5aa726317d4d9775bafc95f3a5fce.jpg → 20250718_113757_311472.jpg
  • 20250718_113757_810288_png.rf.bd4f2fea3fea1f4a7d08e7ea026b6e28.jpg → 20250718_113757_810288.jpg
  • 20250718_113758_379151_png.rf.2c246b86db4f24ebfda2c80f76faa469.jpg → 20250718_113758_379151.jpg
  • 20250718_113758_741257_png.rf.0316493e23aecfa6d061103b0812f434.jpg → 20250718_113758_741257.jpg
  • 20250718_1137

In [3]:
import cv2
import os

def show_annotated_image(img_path, label_path, class_names=None):
    """
    img_path:   Path to the image file (.jpg or .png)
    label_path: Path to the corresponding YOLO .txt (same base name)
    class_names: optional list mapping class_id -> class_name
    """
    # 1. Load image
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # 2. Read YOLO labels
    with open(label_path, 'r') as f:
        lines = [l.strip().split() for l in f if l.strip()]

    # 3. Draw each box
    for parts in lines:
        cls_id, x_c, y_c, bw, bh = map(float, parts)
        # Convert normalized coords back to pixels
        x_c *= w;  y_c *= h
        bw  *= w;  bh  *= h
        x1 = int(x_c - bw/2)
        y1 = int(y_c - bh/2)
        x2 = int(x_c + bw/2)
        y2 = int(y_c + bh/2)

        # Draw rectangle
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)

        # Label text
        if class_names:
            label = class_names[int(cls_id)]
        else:
            label = str(int(cls_id))
        cv2.putText(img, label, (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

    # 4. Show
    cv2.imshow(f"Annotated: {os.path.basename(img_path)}", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__ == "__main__":
    # Example usage:
    # Suppose you have:
    #   images/20250718_113752_269634.jpg
    #   labels/20250718_113752_269634.txt
    # and also
    #   images/20250718_113752_269634.png
    #   labels/20250718_113752_269634.txt
    #
    # Just call this function on either file:
    img_file   = "Processed_data/depth/20250718_122757_048892.png"
    label_file = "Processed_data/labels/20250718_122757_048892.txt"
    show_annotated_image(img_file, label_file)

    # # Or for the depth frame:
    # img_file   = "images/20250718_113752_269634.png"
    # label_file = "labels/20250718_113752_269634.txt"
    # show_annotated_image(img_file, label_file)


In [6]:
import cv2
import os

def show_annotated_image(
    img_path,
    label_path,
    class_names=None,
    visualize_depth_hue=False
):
    """
    img_path:   .jpg RGB or .png depth (16-bit)
    label_path: corresponding YOLO .txt
    visualize_depth_hue: if True, render a hue colormap on your depth frame
    """
    # 1) Load the image (depth vs. color)
    if visualize_depth_hue:
        # load 16-bit depth
        depth = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)  # uint16
        if depth is None:
            raise FileNotFoundError(f"Cannot load {img_path}")
        # normalize to 0–255 for colormap
        max_val = float(depth.max() or 1)
        depth8 = cv2.convertScaleAbs(depth, alpha=255.0/max_val)
        # apply hue colormap
        img = cv2.applyColorMap(depth8, cv2.COLORMAP_HSV)
    else:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # BGR uint8
        if img is None:
            raise FileNotFoundError(f"Cannot load {img_path}")

    h, w = img.shape[:2]

    # 2) Read YOLO labels
    with open(label_path, 'r') as f:
        entries = [l.strip().split() for l in f if l.strip()]

    # 3) Draw boxes and labels
    for parts in entries:
        cls_id, x_c, y_c, bw, bh = map(float, parts)
        # to pixels
        x_c *= w;  y_c *= h
        bw  *= w;  bh  *= h
        x1 = int(x_c - bw/2)
        y1 = int(y_c - bh/2)
        x2 = int(x_c + bw/2)
        y2 = int(y_c + bh/2)

        # choose contrasting color (white)
        color = (255, 255, 255)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

        label = class_names[int(cls_id)] if class_names else str(int(cls_id))
        cv2.putText(
            img, label, (x1, y1 - 5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1
        )

    # 4) Show
    win = f"Annotated: {os.path.basename(img_path)}"
    cv2.imshow(win, img)
    cv2.waitKey(0)
    cv2.destroyWindow(win)


if __name__ == "__main__":
    # RGB example (no hue):
    # show_annotated_image(
    #     "Processed_data/rgb/20250718_122757_048892.jpg",
    #     "Processed_data/labels/20250718_122757_048892.txt",
    #     visualize_depth_hue=False
    # )

    # Depth example (with hue):
    show_annotated_image(
        "Processed_data/depth/20250718_122757_048892.png",
        "Processed_data/labels/20250718_122757_048892.txt",
        visualize_depth_hue=True
    )


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'Annotated: 20250718_122757_048892.png' in function 'cvDestroyWindow'


In [ ]:
# predict_images.py

import os
from pathlib import Path
import cv2
import torch
import numpy as np
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# ─── Helper functions ───────────────────────────────────────────────────────────

def load_rgb(path: Path) -> np.ndarray:
    img = cv2.imread(str(path), cv2.IMREAD_COLOR)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def load_depth(path: Path) -> np.ndarray:
    # Must be the matching 16-bit depth map
    d = cv2.imread(str(path), cv2.IMREAD_UNCHANGED).astype(np.float32)
    return (d / (d.max() + 1e-6) * 255).astype(np.uint8)

def get_model(num_classes: int, weights_path: str):
    # Build the 4-channel Faster R-CNN and load weights
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    # patch conv1
    conv1 = model.backbone.body.conv1
    w_old = conv1.weight.clone()
    conv1_new = torch.nn.Conv2d(4, conv1.out_channels,
        kernel_size=conv1.kernel_size, stride=conv1.stride,
        padding=conv1.padding, bias=False)
    with torch.no_grad():
        conv1_new.weight[:, :3] = w_old
        conv1_new.weight[:, 3:].zero_()
    model.backbone.body.conv1 = conv1_new
    # swap predictor
    in_feats = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_feats, num_classes)
    # patch normalization
    model.transform.image_mean = [0.485,0.456,0.406,0.0]
    model.transform.image_std  = [0.229,0.224,0.225,1.0]
    # load weights
    state = torch.load(weights_path, map_location="cpu")
    model.load_state_dict(state)
    model.eval()
    return model

def draw_predictions(rgb, preds, label_map, score_thresh=0.5):
    out = rgb.copy()
    h, w = rgb.shape[:2]
    for box, score, cid in zip(preds["boxes"], preds["scores"], preds["labels"]):
        if score < score_thresh: continue
        x0, y0, x1, y1 = map(int, box.tolist())
        cls_name = label_map.get(int(cid), str(int(cid)))
        cv2.rectangle(out, (x0,y0), (x1,y1), (0,255,0), 2)
        cv2.putText(out,
            f"{cls_name}:{score:.2f}",
            (x0, y0-5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5, (0,255,0), 1,
            cv2.LINE_AA)
    return out

# ─── Main ─────────────────────────────────────────────────────────────────────

if __name__ == "__main__":
    # Adjust these paths and params:
    ROOT = Path("Processed_data")
    WEIGHTS = "best_fasterrcnn_4ch.pth"
    NUM_CLASSES = 6 + 1   # your #classes + background
    SCORE_THRESH = 0.5

    # Map your class IDs → names
    label_map = {
        0: "person",
        1: "car",
        2: "bike",
        3: "truck",
        4: "light",
        5: "sign"
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(NUM_CLASSES, WEIGHTS).to(device)

    for rgb_path in sorted((ROOT/"rgb").glob("*.jpg")):
        base = rgb_path.stem
        depth_path = ROOT/"depth"/f"{base}.png"

        rgb = load_rgb(rgb_path)
        depth = load_depth(depth_path)[..., None]
        fused = np.concatenate([rgb.astype(np.float32)/255.0,
                                depth.astype(np.float32)/255.0],
                                axis=2)
        inp = torch.from_numpy(fused.transpose(2,0,1)).to(device).unsqueeze(0)

        with torch.no_grad():
            preds = model(inp)[0]

        vis = draw_predictions(rgb, preds, label_map, SCORE_THRESH)
        # show or save:
        cv2.imshow("pred", cv2.cvtColor(vis, cv2.COLOR_RGB2BGR))
        cv2.waitKey(0)
        # Or save:
        # cv2.imwrite(f"results/{base}_pred.png", cv2.cvtColor(vis, cv2.COLOR_RGB2BGR))

    cv2.destroyAllWindows()
